<a href="https://colab.research.google.com/github/ShahinGanbar/GPT_from_scratch/blob/main/GPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -L -o input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  3504k      0 --:--:-- --:--:-- --:--:-- 3513k


In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# checking all possible elements in the vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(' '.join(chars))
print(vocab_size)


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
65


In [7]:
# text encoding and decoding
str_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_str = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_str[i] for i in l])

print(encode("Hey,what's up?"))
print(decode([20, 43, 63, 6, 61, 46, 39, 58, 5, 57, 1, 59, 54, 12]
))

[20, 43, 63, 6, 61, 46, 39, 58, 5, 57, 1, 59, 54, 12]
Hey,what's up?


In [8]:
# Encoding the entire text dataset and storing it in a torch tensor

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# Splitting the data into Train and Test sets

n = int(0.9 * len(data))

train_data = data[:n]
test_data = data[n:]

In [10]:
# Defining batch size for loading the data

block_size = 8
train_data[:block_size + 1]



tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f"When the input is {context} the target is {target}")

When the input is tensor([18]) the target is 47
When the input is tensor([18, 47]) the target is 56
When the input is tensor([18, 47, 56]) the target is 57
When the input is tensor([18, 47, 56, 57]) the target is 58
When the input is tensor([18, 47, 56, 57, 58]) the target is 1
When the input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [12]:
torch.manual_seed(1337)
batch_size = 4 # the number of sequences will be processed in parallel
block_size = 8 # maximum length of predictions

def get_batch(split):
  data = train_data if split == 'train' else test_data
  ix = torch.randomint(len(data)-block_size,(batch_size,))
